In [109]:
import sys,os
import numpy as np
import pylab as py
# from corelib_AA import JAMLIB

In [124]:
import sys,os
import numpy as np
import cPickle
from scipy.interpolate import RectBivariateSpline
from tools import load,BAR,logo
 
class JAMLIB(object):
  
  def __init__(self,path):
    logo()
    self.setup_alphaS(order='NLO')
    self.load_tables(path)

  # alphaS

  def setup_alphaS(self,order='NLO'):

    if   order=='LO':  self.order=0
    elif order=='NLO': self.order=1

    self.aZ  = 0.118 /(4*np.pi)
    self.mc2 = 1.43**2        
    self.mb2 = 4.3**2
    self.mZ2 = 91.187**2
    self.mt2 = 172.9**2

    self.beta=np.zeros((7,3))
    for Nf in range(3,7): 
      self.beta[Nf,0]=11.0-2.0/3.0*Nf 
      self.beta[Nf,1]=102.-38.0/3.0*Nf 
      self.beta[Nf,2]=2857.0/2.0-5033.0/18.0*Nf+325.0/54.0*Nf**2 

    self.Q20=1.0
    # uses alphaS(mZ)--> backwards evolution
    self.ab=self.evolve_a(self.mZ2,self.aZ,self.mb2,5)
    self.at=self.evolve_a(self.mZ2,self.aZ,self.mt2,5)
    self.ac=self.evolve_a(self.mb2,self.ab,self.mc2,4)
    self.a0=self.evolve_a(self.mc2,self.ac,self.Q20,3)

    # we will store all Q2 values of alphaS 
    self.storage={}

  def get_Nf(self,Q2):
    Nf=3
    if Q2>=(4.*self.mc2): Nf+=1
    if Q2>=(4.*self.mb2): Nf+=1
    if Q2>=(4.*self.mt2): Nf+=1
    return Nf

  def beta_func(self,a,Nf):
    betaf = -self.beta[Nf,0]
    if self.order>=1: betaf+=-a*self.beta[Nf,1]
    if self.order>=2: betaf+=-a*self.beta[Nf,2]
    return betaf*a**2

  def evolve_a(self,Q20,a,Q2,Nf):
    # Runge-Kutta implemented in pegasus  
    LR = np.log(Q2/Q20)/20.0
    for k in range(20):
      XK0 = LR * self.beta_func(a,Nf)
      XK1 = LR * self.beta_func(a + 0.5 * XK0,Nf)
      XK2 = LR * self.beta_func(a + 0.5 * XK1,Nf)
      XK3 = LR * self.beta_func(a + XK2,Nf)
      a+= (XK0 + 2.* XK1 + 2.* XK2 + XK3) * 0.166666666666666
    return a

  def get_a(self,Q2):

    if Q2 in self.storage:
      return self.storage[Q2]
    else:
      if self.mt2<=Q2:
        Q20,a0,Nf=self.mt2,self.at,6
      elif self.mb2<=Q2 and Q2<self.mt2: 
        Q20,a0,Nf=self.mb2,self.ab,5
      elif self.mc2<=Q2 and Q2<self.mb2: 
        Q20,a0,Nf=self.mc2,self.ac,4
      elif Q2<self.mc2:
        Q20,a0,Nf=self.mc2,self.ac,3
      a=self.evolve_a(Q20,a0,Q2,Nf)
      self.storage[Q2]=a
      return a

  def get_alphaS(self,Q2):
    return self.get_a(Q2)*4*np.pi

  # interpolation for xF

  def load_tables(self,path):
    TAB=[]
    F=os.listdir(path)
    self.npos=len(F)
    bar=BAR('loading tables',len(F))
    for f in F:
      tab=load('%s/%s'%(path,f)) 
      X=tab['X']
      Q2=tab['Q2']
      for k in tab:
        if k=='X' or k=='Q2': continue
        for kk in tab[k]:
          tab[k][kk]=RectBivariateSpline(X,Q2,tab[k][kk])
      TAB.append(tab)
      bar.next()
    bar.finish()
    self.TAB=TAB
    self.Xgrid = X
    self.Q2grid = Q2

  def get_XF(self,ipos,dist,flav,x,Q2):
    return self.TAB[ipos][dist][flav](x,Q2)[0,0]

  def get_Xgrid(self):
    return self.Xgrid

  def get_Q2grid(self):
    return self.Q2grid

In [197]:
jamFF=JAMLIB('FFpions')

 ################################################ 
                                                 
     _   _    __  __ _     ___ ____              
    | | / \  |  \/  | |   |_ _| __ )             
 _  | |/ _ \ | |\/| | |    | ||  _ \             
| |_| / ___ \| |  | | |___ | || |_) |            
 \___/_/   \_\_|  |_|_____|___|____/             
                                                 
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~  
                                                 
Authors:                                         
Nobuo Sato   nsato@jlab.org                      
Jake Ethier                                      
Wally Melnitchouk                                
Alberto Accardi                                  
################################################ 
loading tables [99%]


In [196]:
x,Q2=0.5,10.0

print 'alphaS     = ',jamFF.get_alphaS(Q2)
print 'num pos    = ',jamFF.npos
print 'xF(ipos=0) = ',jamFF.get_XF(0,'FFpion','up',x,Q2)

Xmin = np.min(jamFF.get_Xgrid())
Xmax = np.max(jamFF.get_Xgrid())
print 'Xmin,Xmax: ', xmin,xmax

Q2min = np.min(jamFF.get_Q2grid())
Q2max = np.max(jamFF.get_Q2grid())
print 'Q2min,Q2max: ', Q2min,Q2max

alphaS     =  0.24654958796
num pos    =  200
xF(ipos=0) = 

KeyError: 'FFpion'

## Info file

In [173]:
# Information for .info file headers
desc = 'JAMFF16 pion fragmentation functions. \
alphas(MZ)=0.118. mem=0 => average of posteriors; mem=1-200 => FF posteriors.'
auth = "[JAM collaboration] Sato N, Ethier J J, Melnitchouk W, Hirai M, Kumano S, Accardi A"
ref = "arXiv:1609.xxxx - e-mail accardi@jlab.org / sato@jlab.org"
index = 16000
    # particle=hadron to which the PDFs belong, see
    # http://pdg.lbl.gov/2015/reviews/rpp2015-rev-monte-carlo-numbering.pdf
    # For protons, neutrons, use format ±n nr nL nq1 nq2 nq3 nJ
    # E.g, proton = 2212 (u; u; d; spin 1/2 --> nJ=2*s+1)
    # For nuclei, use LZZZAAAI, see point 14 in PDG standard:
    # L nonzero is for hypernuclei, I represent isomer levels with
    # I = 0 the ground state, and Z and A should be self-explanatory.
hadron = 2212  #  2212=proton

errortype = 'MC'    # MC = Monte Carlo method --> nsamples = no. of posteriors
nsamples = 200      # HESSIAN = Hessian --> nsamples = no. free parameters
cl = 68.3           # confidence level

# Collects info from JAM fits
xgrid = jamFF.get_Xgrid()
xmin = np.min(xgrid)
xmax = np.max(xgrid)
Qgrid = jamFF.get_Q2grid()**0.5
Qmin = np.min(Qgrid)
Qmax = np.max(Qgrid)
mu = 0.
md = 0.
ms = 0.1
mc = 1.34
mb = 4.5
mt = 180.0

In [192]:
H=[]
H.append('SetDesc: '+desc)
H.append('SetIndex: %i'%index)
H.append('Authors: '+auth)
H.append('Reference: '+ref)
H.append('Format: lhagrid1')
H.append('DataVersion: %i'%ver)

H.append('Particle: %i'%hadron)
flavors = [-5, -4, -3, -2, -1, 1, 2, 3, 4, 5, 21]
H.append('Flavors: '+str(flavors))
H.append('OrderQCD: %i'%ordQCD)
H.append('ForcePositive: 1')
flavscheme = 'variable' # used for any variable flavor number scheme, massive or not
H.append('FlavorScheme: '+flavscheme)
H.append('NumFlavors: %i'%nfl)

if errortype=='MC':
  nsets = nsamples+1   # no. posteriors + average
  errtype = 'replicas'  # LHAPDF-speak for MC method
elif errortype=='HESSIAN': 
  nstes = 2*nsamples+2  # 2*eigenvalues + central fit
  errtype = 'hessian'
H.append('NumMembers: %i'%nsets)
H.append('ErrorType: '+errtype)
H.append('ErrorConfLevel: %f'%cl)
H.append('XMin: %e'%xmin)
H.append('XMax: %e'%xmax)
H.append('QMin: %e'%Qmin)
H.append('QMax: %e'%Qmax)

MZ = 91.1876
H.append('MZ: %f'%MZ**2)
H.append('MUp: %f'%mu)
H.append('MDown: %f'%md)
H.append('MStrange: %f'%ms)
H.append('MCharm: %f'%mc)
H.append('MBottom: %f'%mb)
H.append('MTop: %f'%mt)

H.append('AlphaS_MZ: %f'%jamFF.get_alphaS(MZ**2))
H.append('AlphaS_OrderQCD: %f'%ordQCD)
H.append('AlphaS_Type: ipol')
H.append(Qgrid**2)    #  <--------------  check small Q values, grid is way too coarse [Nobuo will do this]
H.append('AlphaS_Qs: '+str(Qgrid.tolist()))
alphas = []
for Q in Qgrid:
  alphas.append(jamFF.get_alphaS(Q**2))
H.append('AlphaS_Vals: '+str(alphas))
H.append('AlphaS_Lambda4: %f'%jamFF.get_alphaS(mc**2))
H.append('AlphaS_Lambda5: %f'%jamFF.get_alphaS(mb**2))
   

for l in H: print l 

SetDesc: JAMFF16 pion fragmentation functions. alphas(MZ)=0.118. mem=0 => average of posteriors; mem=1-200 => FF posteriors.
SetIndex: 16000
Authors: [JAM collaboration] Sato N, Ethier J J, Melnitchouk W, Hirai M, Kumano S, Accardi A
Reference: arXiv:1609.xxxx - e-mail accardi@jlab.org / sato@jlab.org
Format: lhagrid1
DataVersion: -1
Particle: 2212
Flavors: [-5, -4, -3, -2, -1, 1, 2, 3, 4, 5, 21]
OrderQCD: 1
ForcePositive: 1
FlavorScheme: variable
NumFlavors: 5
NumMembers: 201
ErrorType: replicas
ErrorConfLevel: 68.300000
XMin: 1.000000e-06
XMax: 1.000000e+00
QMin: 1.000000e+00
QMax: 3.162278e+02
MZ: 8315.178394
MUp: 0.000000
MDown: 0.000000
MStrange: 0.100000
MCharm: 1.340000
MBottom: 4.500000
MTop: 180.000000
AlphaS_MZ: 0.118000
AlphaS_OrderQCD: 1.000000
AlphaS_Type: ipol
[  1.00000000e+00   3.44924138e+03   6.89748276e+03   1.03457241e+04
   1.37939655e+04   1.72422069e+04   2.06904483e+04   2.41386897e+04
   2.75869310e+04   3.10351724e+04   3.44834138e+04   3.79316552e+04
   4.137

In [208]:
print jamFF.get_XF(0,'FFpion','up',x,Q2)

0.458797885132


## write single .dat file

In [226]:
def writedat(ipos,hadron):
  
  if ipos==0: 
    type='central'
  else:
    type='replica'

  H = []
  H.append('PdfType: '+type)
  H.append('Format: lhagrid1')
  H.append('---')  
  
  for l in H: print l
  
  xvals = ["%10.4e" % x for x in xgrid]
  print ' '.join(xvals)
  Qvals = ["%10.4e" % Q for Q in Qgrid]
  print ' '.join(Qvals)
  flavs = ["%2i" % f for f in flavors]
  print ' '.join(flavs)
    
  partons = ['bp','cp','sp','up','dp','dp','up','sp','cp','bp','g']    #   <----------  add the others
  npart = len(partons)
  
  full_dist = []
  for x in xgrid:
    for Q in Qgrid:
      for f in partons:
        FF = jamFF.get_XF(ipos,hadron,'up',x,Q**2)    # <-----  FF or x*FF ??  (Nobuo returns x*FF, check what is standard in LHAPDF)
        #if x==1.: FF = 0.  
        full_dist.append(FF)
  template = ''.join(['{%i:'%n+'%12.5e} ' for n in range(npart)])    # <----- how many sig digits ?
  print template.format(full_dist)
        
    
hadron = 'FFpion'
writedat(0,hadron)

PdfType: central
Format: lhagrid1
---
1.0000e-06 1.0102e-02 2.0203e-02 3.0304e-02 4.0405e-02 5.0506e-02 6.0607e-02 7.0708e-02 8.0809e-02 9.0910e-02 1.0101e-01 1.1111e-01 1.2121e-01 1.3131e-01 1.4141e-01 1.5152e-01 1.6162e-01 1.7172e-01 1.8182e-01 1.9192e-01 2.0202e-01 2.1212e-01 2.2222e-01 2.3232e-01 2.4243e-01 2.5253e-01 2.6263e-01 2.7273e-01 2.8283e-01 2.9293e-01 3.0303e-01 3.1313e-01 3.2323e-01 3.3333e-01 3.4343e-01 3.5354e-01 3.6364e-01 3.7374e-01 3.8384e-01 3.9394e-01 4.0404e-01 4.1414e-01 4.2424e-01 4.3434e-01 4.4444e-01 4.5455e-01 4.6465e-01 4.7475e-01 4.8485e-01 4.9495e-01 5.0505e-01 5.1515e-01 5.2525e-01 5.3535e-01 5.4545e-01 5.5556e-01 5.6566e-01 5.7576e-01 5.8586e-01 5.9596e-01 6.0606e-01 6.1616e-01 6.2626e-01 6.3636e-01 6.4647e-01 6.5657e-01 6.6667e-01 6.7677e-01 6.8687e-01 6.9697e-01 7.0707e-01 7.1717e-01 7.2727e-01 7.3737e-01 7.4748e-01 7.5758e-01 7.6768e-01 7.7778e-01 7.8788e-01 7.9798e-01 8.0808e-01 8.1818e-01 8.2828e-01 8.3838e-01 8.4849e-01 8.5859e-01 8.6869e-01 8.787

ValueError: Invalid conversion specification